### **Análise de Sentimentos sobre Estados Emocionais: Explorando o Poder das Palavras**

Nesta análise, nos propusemos a compreender e prever os estados emocionais de indivíduos com base em declarações textuais, utilizando técnicas de **Machine Learning** e **Processamento de Linguagem Natural (NLP)**. Em um mundo onde as emoções humanas são cada vez mais expressas em forma digital — seja por meio de redes sociais, blogs, ou até mesmo em mensagens de texto — a capacidade de identificar e categorizar esses sentimentos é crucial para inúmeras aplicações, desde suporte psicológico até aprimoramento de interações virtuais.

Nosso dataset foi composto por milhares de declarações, cada uma rotulada com um estado emocional como **Anxiety**, **Depression**, **Stress**, entre outros. Utilizamos técnicas de limpeza de texto para remover ruídos e padronizar os dados, e em seguida aplicamos o **TF-IDF** para transformar essas palavras em vetores numéricos que pudessem ser usados por um modelo de **Regressão Logística**.

Com uma acurácia inicial de **74,4%**, o modelo demonstrou uma capacidade notável de prever corretamente os estados emocionais, destacando-se em classes mais comuns como "Normal" e "Anxiety", enquanto enfrentou desafios com classes minoritárias, como "Personality disorder" e "Stress". Ao aplicar técnicas de balanceamento como o **SMOTE**, conseguimos melhorar o reconhecimento de classes menos representadas, embora com um impacto na precisão geral.

A análise foi concluída com a aplicação do modelo em novas frases geradas, permitindo uma avaliação da robustez e aplicabilidade do modelo em novos dados.

Este projeto exemplifica como a combinação de técnicas de NLP e aprendizado de máquina pode ser poderosa para entender os sentimentos expressos em texto, abrindo portas para diversas aplicações no campo da saúde mental, comunicação digital, e além.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('sentimental.csv')

In [3]:
df.head()

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


## Pré-processamento do texto

In [7]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Função de limpeza de texto
def clean_text(text):
    if isinstance(text, str):  # Verificar se o texto é uma string
        # Remover pontuações
        text = re.sub(r'[^\w\s]', '', text)
        # Colocar em minúsculas
        text = text.lower()
        # Remover stopwords
        stop_words = set(stopwords.words('english'))
        words = text.split()
        words = [word for word in words if word not in stop_words]
        # Lematização
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]
        return ' '.join(words)
    else:
        return ''  # Retornar string vazia se o valor não for uma string

# Aplicar a limpeza ao DataFrame
df['cleaned_statement'] = df['statement'].apply(clean_text)

# Exibir as primeiras linhas para ver o resultado
df.head()

,Unnamed: 0,statement,status,cleaned_statement
0,0,oh my gosh,Anxiety,oh gosh
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleeping confused mind restless heart ...
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,wrong back dear forward doubt stay restless re...
3,3,I've shifted my focus to something else but I'...,Anxiety,ive shifted focus something else im still worried
4,4,"I'm restless and restless, it's been a month n...",Anxiety,im restless restless month boy mean


## Transformação o texto limpo em vetores TF-IDF:

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializar o vetorizador TF-IDF
tfidf = TfidfVectorizer(max_features=1000)  # Limitando o número de features a 1000

# Ajustar o TF-IDF nos textos e transformá-los
X = tfidf.fit_transform(df['cleaned_statement'])

# Verificar o shape da matriz resultante
print(X.shape)  # Deve ser (número de linhas, 1000 features)

# Exibir as primeiras features geradas
print(tfidf.get_feature_names_out()[:10])

(53043, 1000)
['10' '100' '12' '15' '16' '17' '18' '20' '21' '23']


## Modelo e Treinamento

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, df['status'], test_size=0.2, random_state=42)

# Inicializar o modelo de Regressão Logística
model = LogisticRegression(max_iter=1000)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7439909510792723
Classification Report:
                       precision    recall  f1-score   support

             Anxiety       0.75      0.70      0.73       779
             Bipolar       0.81      0.65      0.72       580
          Depression       0.70      0.71      0.71      3100
              Normal       0.82      0.95      0.88      3327
Personality disorder       0.83      0.42      0.56       248
              Stress       0.68      0.43      0.52       557
            Suicidal       0.65      0.62      0.64      2018

            accuracy                           0.74     10609
           macro avg       0.75      0.64      0.68     10609
        weighted avg       0.74      0.74      0.74     10609



O modelo alcançou uma **acurácia de 74,4%**, o que é um bom ponto de partida. No entanto, ao observar o **relatório de classificação**, vemos que:

1. **Classes como "Normal"** apresentam alta precisão e recall, o que é esperado, já que podem ser mais representadas no dataset.
2. **Classes menores como "Personality disorder" e "Stress"** têm um desempenho inferior, especialmente no recall, sugerindo que o modelo tem mais dificuldade para identificar esses casos.

#### Vamos aplicar o SMOTE (Synthetic Minority Over-sampling Technique) para aumentar a representação das classes minoritárias no conjunto de treino

In [10]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, df['status'], test_size=0.2, random_state=42)

# Aplicar o SMOTE no conjunto de treino
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Inicializar o modelo de Regressão Logística
model = LogisticRegression(max_iter=1000)

# Treinar o modelo com os dados balanceados
model.fit(X_train_smote, y_train_smote)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6633990008483364
Classification Report:
                       precision    recall  f1-score   support

             Anxiety       0.61      0.75      0.67       779
             Bipolar       0.57      0.72      0.64       580
          Depression       0.76      0.61      0.68      3100
              Normal       0.86      0.69      0.76      3327
Personality disorder       0.25      0.61      0.35       248
              Stress       0.37      0.56      0.45       557
            Suicidal       0.60      0.69      0.64      2018

            accuracy                           0.66     10609
           macro avg       0.57      0.66      0.60     10609
        weighted avg       0.71      0.66      0.68     10609



#### Após aplicar o SMOTE, observamos que a acurácia geral caiu para 66,3%, mas o desempenho nas classes minoritárias, como "Personality disorder" e "Stress", melhorou consideravelmente, especialmente no recall. Isso é comum ao aplicar o SMOTE, pois o modelo foca mais em aprender as classes menos representadas, mas pode comprometer a precisão geral.

#### Vamos usar o primeiro modelo com acurácia de 74,4% e manter o modelo de Regressão Logística sem o uso de SMOTE para praticar com o modelo.

## Modelo na Práctica

Essas 20 frases podem ser usadas para testar o modelo, cobrindo uma variedade de sentimentos e possíveis estados emocionais:

1. "I'm feeling extremely overwhelmed with everything happening around me."
2. "I had a fantastic day today, everything just went perfectly!"
3. "It's been really tough lately, I feel like I'm losing control."
4. "I'm content and peaceful, just enjoying the little moments."
5. "My thoughts have been all over the place, and I can't focus on anything."
6. "I feel so down and unmotivated, I don’t know what to do next."
7. "I'm so angry and frustrated, nothing is going as planned."
8. "I have this weird sensation of calm, even though things aren't perfect."
9. "It feels like I'm stuck in a never-ending cycle of sadness."
10. "Everything is going smoothly at work, I feel really accomplished."
11. "My emotions are a rollercoaster right now, up and down all day."
12. "I can’t stop worrying about the future, it’s giving me so much anxiety."
13. "I’m feeling really proud of myself for finally achieving my goals."
14. "It feels like I’m living in a cloud, disconnected from everything."
15. "Today was one of the best days I’ve had in a long time!"
16. "I don’t feel anything at all, just completely numb."
17. "I’m excited about the future and what it holds for me."
18. "My mind is racing with negative thoughts, I just want it to stop."
19. "I feel happy, but there's this underlying tension I can't shake."
20. "I've been really calm and composed, handling stress well lately."

In [11]:
# Lista de frases geradas
test_phrases = [
    "I'm feeling extremely overwhelmed with everything happening around me.",
    "I had a fantastic day today, everything just went perfectly!",
    "It's been really tough lately, I feel like I'm losing control.",
    "I'm content and peaceful, just enjoying the little moments.",
    "My thoughts have been all over the place, and I can't focus on anything.",
    "I feel so down and unmotivated, I don’t know what to do next.",
    "I'm so angry and frustrated, nothing is going as planned.",
    "I have this weird sensation of calm, even though things aren't perfect.",
    "It feels like I'm stuck in a never-ending cycle of sadness.",
    "Everything is going smoothly at work, I feel really accomplished.",
    "My emotions are a rollercoaster right now, up and down all day.",
    "I can’t stop worrying about the future, it’s giving me so much anxiety.",
    "I’m feeling really proud of myself for finally achieving my goals.",
    "It feels like I’m living in a cloud, disconnected from everything.",
    "Today was one of the best days I’ve had in a long time!",
    "I don’t feel anything at all, just completely numb.",
    "I’m excited about the future and what it holds for me.",
    "My mind is racing with negative thoughts, I just want it to stop.",
    "I feel happy, but there's this underlying tension I can't shake.",
    "I've been really calm and composed, handling stress well lately."
]

# Aplicar a limpeza nas frases geradas
cleaned_test_phrases = [clean_text(phrase) for phrase in test_phrases]

# Converter as frases em TF-IDF
X_test_phrases = tfidf.transform(cleaned_test_phrases)

# Fazer previsões
predictions = model.predict(X_test_phrases)

# Exibir as previsões
for phrase, prediction in zip(test_phrases, predictions):
    print(f"Frase: {phrase}\nPrevisão: {prediction}\n")

Frase: I'm feeling extremely overwhelmed with everything happening around me.
Previsão: Stress

Frase: I had a fantastic day today, everything just went perfectly!
Previsão: Normal

Frase: It's been really tough lately, I feel like I'm losing control.
Previsão: Bipolar

Frase: I'm content and peaceful, just enjoying the little moments.
Previsão: Personality disorder

Frase: My thoughts have been all over the place, and I can't focus on anything.
Previsão: Stress

Frase: I feel so down and unmotivated, I don’t know what to do next.
Previsão: Stress

Frase: I'm so angry and frustrated, nothing is going as planned.
Previsão: Stress

Frase: I have this weird sensation of calm, even though things aren't perfect.
Previsão: Anxiety

Frase: It feels like I'm stuck in a never-ending cycle of sadness.
Previsão: Bipolar

Frase: Everything is going smoothly at work, I feel really accomplished.
Previsão: Depression

Frase: My emotions are a rollercoaster right now, up and down all day.
Previsão: St

### **Conclusão: A Importância e os Benefícios da Análise de Sentimentos em Estados Emocionais**

A análise de sentimentos aplicada a estados emocionais representa uma poderosa ferramenta para a compreensão e intervenção em questões relacionadas à saúde mental e bem-estar. Neste projeto, fomos capazes de construir um modelo capaz de interpretar declarações textuais e identificar os sentimentos por trás delas com um alto grau de precisão. Essa capacidade de categorizar emoções a partir de texto oferece uma série de benefícios importantes.

Em primeiro lugar, essa análise pode ser aplicada em plataformas de suporte psicológico ou terapêutico digital, permitindo que sistemas automáticos identifiquem sinais de alerta, como **ansiedade** ou **depressão**, em tempo real. Isso poderia proporcionar uma intervenção precoce e o suporte necessário a indivíduos em momentos críticos, onde uma resposta rápida pode fazer toda a diferença.

Além disso, o uso dessas técnicas em plataformas de redes sociais ou sistemas de feedback de usuários pode ajudar a compreender melhor os sentimentos de grandes grupos, fornecendo insights valiosos sobre o humor coletivo, os desafios emocionais enfrentados, e até mesmo as tendências psicológicas emergentes em diferentes momentos e contextos. Em um cenário de negócios, isso poderia guiar a tomada de decisões mais informadas sobre produtos, marketing e atendimento ao cliente.

A integração de técnicas como **SMOTE** demonstrou como o balanceamento de dados pode ajudar a melhorar a capacidade do modelo de identificar estados emocionais mais raros, algo fundamental em ambientes onde a diversidade emocional é ampla e complexa.

Em resumo, este trabalho ressalta a importância de análises de sentimentos em diversos contextos e os benefícios que elas trazem para áreas como a saúde mental, atendimento ao cliente, e comunicação digital. A habilidade de compreender e responder aos sentimentos expressos por meio de texto é uma fronteira poderosa da inteligência artificial, oferecendo uma maneira eficaz de humanizar e melhorar as interações digitais.